Dependencies

In [ ]:
!pip install --upgrade --force-reinstall numpy==1.26.4 scikit-learn==1.6.0 tqdm==4.67.1 ultralytics==8.3.0 opencv-python-headless==4.10.0.84 python-docx==1.1.2 pyyaml==6.0.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.7/109.7 kB 11.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 116.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.3/881.3 kB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/24

In [ ]:
import os, shutil, glob, random, json, yaml, re
import numpy as np
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from ultralytics import YOLO

# reproducibility
random.seed(42)

# ---- CHANGE these paths to match where you placed the folders in Drive ----
SOURCE_IMAGES_DIR = "/content/drive/MyDrive/Tooth_Dataset/images"   # <-- edit if different
SOURCE_LABELS_DIR = "/content/drive/MyDrive/Tooth_Dataset/labels"   # <-- edit if different

BASE = "/content/oralvis"
DATASET_DIR = f"{BASE}/dataset"
YAML_PATH    = f"{BASE}/data.yaml"
RUNS_DIR     = f"{BASE}/runs"
SUBMIT_DIR   = f"{BASE}/submission"
PRED_DIR     = f"{BASE}/pred_samples"

for d in [BASE, DATASET_DIR, RUNS_DIR, SUBMIT_DIR, PRED_DIR]:
    os.makedirs(d, exist_ok=True)

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp"}
print("Drive mounted & paths set ✅")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Drive mounted & paths set ✅


In [ ]:
def find_pairs(images_dir, labels_dir):
    images, labels = [], []
    for img_path in glob.glob(os.path.join(images_dir, "*")):
        ext = os.path.splitext(img_path)[1].lower()
        if ext in IMG_EXTS:
            stem = os.path.splitext(os.path.basename(img_path))[0]
            lab_path = os.path.join(labels_dir, stem + ".txt")
            if os.path.exists(lab_path):   # only pick txt labels
                images.append(img_path)
                labels.append(lab_path)
    return images, labels

# Get pairs
images, labels = find_pairs(SOURCE_IMAGES_DIR, SOURCE_LABELS_DIR)
print(f"Found {len(images)} images with matching labels")

# Split 80/10/10
X_train, X_tmp, y_train, y_tmp = train_test_split(images, labels, test_size=0.2, random_state=42)
X_val,   X_test, y_val, y_test = train_test_split(X_tmp, y_tmp, test_size=0.5, random_state=42)

# Make split dirs
for sp in ["train","val","test"]:
    os.makedirs(f"{DATASET_DIR}/images/{sp}", exist_ok=True)
    os.makedirs(f"{DATASET_DIR}/labels/{sp}", exist_ok=True)

# Copy files
def copy_files(imgs, labs, split):
    for src in tqdm(imgs, desc=f"Copying {split} images"):
        shutil.copy2(src, f"{DATASET_DIR}/images/{split}")
    for src in tqdm(labs, desc=f"Copying {split} labels"):
        shutil.copy2(src, f"{DATASET_DIR}/labels/{split}")

copy_files(X_train, y_train, "train")
copy_files(X_val,   y_val,   "val")
copy_files(X_test,  y_test,  "test")

print("✅ Split complete:", len(X_train), "train,", len(X_val), "val,", len(X_test), "test")


Found 497 images with matching labels


Copying test labels: 100%|██████████| 50/50 [00:32<00:00,  1.55it/s]

✅ Split complete: 397 train, 50 val, 50 test


In [ ]:
CLASS_NAMES = [
    "Canine (13)", "Canine (23)", "Canine (33)", "Canine (43)",
    "Central Incisor (21)", "Central Incisor (41)", "Central Incisor (31)", "Central Incisor (11)",
    "First Molar (16)", "First Molar (26)", "First Molar (36)", "First Molar (46)",
    "First Premolar (14)", "First Premolar (34)", "First Premolar (44)", "First Premolar (24)",
    "Lateral Incisor (22)", "Lateral Incisor (32)", "Lateral Incisor (42)", "Lateral Incisor (12)",
    "Second Molar (17)", "Second Molar (27)", "Second Molar (37)", "Second Molar (47)",
    "Second Premolar (15)", "Second Premolar (25)", "Second Premolar (35)", "Second Premolar (45)",
    "Third Molar (18)", "Third Molar (28)", "Third Molar (38)", "Third Molar (48)"
]

DATA_YAML = {
    "path": DATASET_DIR,
    "train": "images/train",
    "val":   "images/val",
    "test":  "images/test",
    "names": {i: n for i, n in enumerate(CLASS_NAMES)}
}

with open(YAML_PATH, "w") as f:
    yaml.safe_dump(DATA_YAML, f, sort_keys=False, allow_unicode=True)

print("✅ data.yaml created at:", YAML_PATH)
print(open(YAML_PATH).read())


✅ data.yaml created at: /content/oralvis/data.yaml
path: /content/oralvis/dataset
train: images/train
val: images/val
test: images/test
names:
  0: Canine (13)
  1: Canine (23)
  2: Canine (33)
  3: Canine (43)
  4: Central Incisor (21)
  5: Central Incisor (41)
  6: Central Incisor (31)
  7: Central Incisor (11)
  8: First Molar (16)
  9: First Molar (26)
  10: First Molar (36)
  11: First Molar (46)
  12: First Premolar (14)
  13: First Premolar (34)
  14: First Premolar (44)
  15: First Premolar (24)
  16: Lateral Incisor (22)
  17: Lateral Incisor (32)
  18: Lateral Incisor (42)
  19: Lateral Incisor (12)
  20: Second Molar (17)
  21: Second Molar (27)
  22: Second Molar (37)
  23: Second Molar (47)
  24: Second Premolar (15)
  25: Second Premolar (25)
  26: Second Premolar (35)
  27: Second Premolar (45)
  28: Third Molar (18)
  29: Third Molar (28)
  30: Third Molar (38)
  31: Third Molar (48)



In [ ]:
!pip uninstall -y wandb


Found existing installation: wandb 0.21.1
Uninstalling wandb-0.21.1:
  Successfully uninstalled wandb-0.21.1


In [ ]:
# Train YOLOv8 model on your dataset
!yolo detect train \
    data={YAML_PATH} \
    model=yolov8s.pt \
    imgsz=640 \
    epochs=100 \
    batch=16 \
    project=oralvis \
    name=tooth_yolo \
    pretrained=True \
    verbose=True


New https://pypi.org/project/ultralytics/8.3.189 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.0 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/oralvis/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=oralvis, name=tooth_yolo2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, sa

In [ ]:
import shutil, os

RUN_DIR = "/content/oralvis/tooth_yolo2"   # 👈 change this to your latest run folder
SUBMIT_DIR = "/content/oralvis/submission"
os.makedirs(SUBMIT_DIR, exist_ok=True)

# Important plots
for fn in ["results.png", "confusion_matrix.png", "PR_curve.png", "F1_curve.png"]:
    src = os.path.join(RUN_DIR, fn)
    if os.path.exists(src):
        shutil.copy2(src, os.path.join(SUBMIT_DIR, fn))

# Best weights
best_w = os.path.join(RUN_DIR, "weights", "best.pt")
if os.path.exists(best_w):
    shutil.copy2(best_w, os.path.join(SUBMIT_DIR, "best.pt"))

print("✅ Submission folder ready")
print("Files now in submission folder:", os.listdir(SUBMIT_DIR))


✅ Submission folder ready
Files now in submission folder: ['PR_curve.png', 'F1_curve.png', 'confusion_matrix.png', 'best.pt', 'results.png']


In [ ]:
from ultralytics import YOLO
import glob, os, shutil

# Load trained model
model = YOLO("/content/oralvis/submission/best.pt")

# Pick a few test images (you can adjust the number)
test_imgs = glob.glob(f"{DATASET_DIR}/images/test/*")[:6]

# Run inference
preds = model.predict(
    source=test_imgs,
    imgsz=640,
    conf=0.25,
    save=True,
    project="/content/oralvis",
    name="pred_samples"
)

# Copy predictions to submission folder
PRED_DIR = preds[0].save_dir if hasattr(preds[0], "save_dir") else "/content/oralvis/pred_samples"
for p in glob.glob(os.path.join(PRED_DIR, "*")):
    if os.path.splitext(p)[1].lower() in [".jpg",".jpeg",".png"]:
        shutil.copy2(p, os.path.join(SUBMIT_DIR, os.path.basename(p)))

print("✅ Sample predictions saved to submission folder")
print("Files in submission now:", os.listdir(SUBMIT_DIR))



0: 640x640 1 Canine (33), 1 Canine (43), 1 Central Incisor (41), 1 Central Incisor (31), 1 First Premolar (34), 1 First Premolar (44), 1 Lateral Incisor (32), 1 Lateral Incisor (42), 1 Second Molar (37), 1 Second Molar (47), 1 Second Premolar (35), 1 Second Premolar (45), 13.9ms
1: 640x640 1 Canine (13), 2 Canine (23)s, 1 Canine (33), 1 Canine (43), 1 Central Incisor (21), 1 Central Incisor (41), 1 Central Incisor (31), 1 Central Incisor (11), 2 First Molar (26)s, 1 First Molar (36), 1 First Molar (46), 1 First Premolar (34), 1 First Premolar (44), 2 First Premolar (24)s, 2 Lateral Incisor (22)s, 1 Lateral Incisor (32), 1 Lateral Incisor (42), 1 Lateral Incisor (12), 2 Second Molar (27)s, 2 Second Molar (37)s, 1 Second Molar (47), 2 Second Premolar (25)s, 2 Second Premolar (35)s, 1 Second Premolar (45), 2 Third Molar (28)s, 2 Third Molar (38)s, 1 Third Molar (48), 13.9ms
2: 640x640 1 Canine (13), 1 Canine (23), 1 Canine (33), 1 Canine (43), 1 Central Incisor (21), 1 Central Incisor (4